In [1]:
import pathlib
import re

In [2]:
class RunResult:
    Serial="Serial"
    OpenMP="OpenMP"
    MPI="MPI"
    
    def __init__(self, name, millis, procCnt, runType, nodeCnt=None):
        self.name = name
        self.millis = millis
        self.procCnt = procCnt
        self.nodeCnt = nodeCnt
        self.runType = runType
            
        
    def __str__(self):
        return f"""
        {self.name} ({self.runType})\n
        \tDuration [ms]: {self.millis}\n
        \tproc: {self.procCnt}\n
        \tnode: {self.nodeCnt}\n
        """
class BaseParser:
    
    @staticmethod 
    def parseStdout(stdout):
        with open(stdout) as file:
            data = file.read().splitlines()
            
        millis = None
        for i, line in enumerate(data):
            if re.match(f'^Cena', line):
                millis = int(data[i+1].split()[2])
                break
                
        return millis

# Serial

In [3]:
class SerialParser(BaseParser):
    
    @staticmethod 
    def parseFilename(filename):
        return 1, str(filename)
    
    @staticmethod 
    def parse(path):
        procCnt, name = SerialParser.parseFilename(path.name)
        millis = SerialParser.parseStdout(path / 'stdout')
        return RunResult(name, millis, procCnt, RunResult.Serial)
    

pSerial = pathlib.Path('serial')
serialLst = []
for p in pSerial.glob('*'):
    serialLst.append(SerialParser.parse(p))
    
serialKilled = set([x.name+'-p'+str(x.procCnt) for x in serialLst if x.millis is None])
print(f'Killed {serialKilled}')

Killed {'saj6-p1'}


# OpenMP

In [4]:
class OpenMpParser(BaseParser):
    
    @staticmethod 
    def parseFilename(filename):
        sp = str(filename).split('-')
        name = sp[0]
        procCnt = int(sp[1][1:])
        return procCnt, name
    
    @staticmethod 
    def parse(path):
        procCnt, name = OpenMpParser.parseFilename(path.name)
        millis = OpenMpParser.parseStdout(path / 'stdout')
        return RunResult(name, millis, procCnt, RunResult.OpenMP)

## Task parallelism

In [5]:
pTaskPar = pathlib.Path('openmp/task-par/out')
taskParLst = []
for p in pTaskPar.glob('*'):
    taskParLst.append(OpenMpParser.parse(p))
    
taskKilled = set([ x.name+'-p'+str(x.procCnt) for x in taskParLst if x.millis is None])
print(f'Killed {taskKilled}')

Killed {'saj12-p2', 'saj12-p16'}


## Task parallelism [threshold]

In [6]:
taskParDict = dict.fromkeys([2, 4, 6, 8, 10, 12])

for t in taskParDict.keys():
    tmp = []
    for p in pathlib.Path(f'openmp/task-par-threshold/out-t{t}').glob('*'):
        tmp.append(OpenMpParser.parse(p))
    taskParDict[t] = tmp
    
print('threshold | killed')
for t in taskParDict.keys():
    print(f'\t{t}: {len([x for x in taskParDict[t] if x.millis is not None])}')

threshold | killed
	2: 13
	4: 6
	6: 18
	8: 11
	10: 12
	12: 9


## Data parallelism 

In [7]:
pDataPar = pathlib.Path('openmp/data-par/out0')
dataParLst = []
for p in pDataPar.glob('*'):
    dataParLst.append(OpenMpParser.parse(p))

dataKilled = set([x.name+'-p'+str(x.procCnt) for x in dataParLst if x.millis is None])
print(f'Killed {dataKilled}')

Killed set()


# MPI

In [8]:
line = "0: Elapsed time is 47.276555."
re.findall(r'\d+.\d+', line)

['47.276555']

In [9]:
class MpiParser(BaseParser):
    
    @staticmethod 
    def parseStdout(stdout):
        with open(stdout) as file:
            data = file.read().splitlines()
            
        millis = None
        for i, line in enumerate(data):
            if re.match(f'^0: Elapsed time is', line):
                millis = int(float(re.findall(r'\d+.\d+', line)[0]) * 1000)
                break
                
        return millis
    
    @staticmethod 
    def parseFilename(filename):
        sp = str(filename).split('-')
        name = sp[0]
        nodeCnt = int(sp[1][1:])
        procCnt = int(sp[2][1:])
        return procCnt, name, nodeCnt
    
    @staticmethod 
    def parse(path):
        procCnt, name, nodeCnt = MpiParser.parseFilename(path.name)
        millis = MpiParser.parseStdout(path / 'stdout')
        return RunResult(name, millis, procCnt, RunResult.MPI, nodeCnt)

In [10]:
mpiLst = []
for p in pathlib.Path('mpi/out0').glob('*'):
    mpiLst.append(MpiParser.parse(p))
    
taskKilled = set([ x.name+'-n'+str(x.nodeCnt)+'-p'+str(x.procCnt) for x in mpiLst if x.millis is None])
taskAlive = set([ x.name+'-p'+str(x.procCnt) for x in mpiLst if x.millis is not None])
print("killed", len(taskKilled), taskKilled, "\n")
print("computed", len(taskAlive), taskAlive)

killed 0 set() 

computed 15 {'saj12-p8', 'saj10-p8', 'saj7-p6', 'saj7-p8', 'saj10-p16', 'saj7-p12', 'saj10-p20', 'saj12-p12', 'saj12-p6', 'saj10-p6', 'saj12-p16', 'saj12-p20', 'saj7-p20', 'saj7-p16', 'saj10-p12'}


# Graphing results

Měřené instance jsou 7, 10, 12.

In [11]:
m = ['saj7', 'saj10', 'saj12']
ser = [x for x in serialLst if x.name in m ]
mpt = [x for x in taskParLst if x.name in m ]
mpd = [x for x in dataParLst if x.name in m ]
mpi = [x for x in mpiLst if x.name in m ]

In [12]:
import pandas as pd

def runResultToList(r):
    return r.runType, r.name, r.millis, r.procCnt, r.nodeCnt
    
lst = []
for r in [item for x in [ser, mpt, mpd, mpi] for item in x]:
    lst.append(runResultToList(r))
df = pd.DataFrame(lst, columns=['type', 'instance', 'millis', 'proc', 'node'])
display(df)

,type,instance,millis,proc,node
0,Serial,saj10,103666.0,1,NaN
1,Serial,saj12,489801.0,1,NaN
2,Serial,saj7,272536.0,1,NaN
3,OpenMP,saj10,131979.0,1,NaN
4,OpenMP,saj10,64735.0,10,NaN
...,...,...,...,...,...
76,MPI,saj7,72642.0,12,4.0
77,MPI,saj7,68517.0,16,4.0
78,MPI,saj7,72602.0,20,4.0
79,MPI,saj7,97800.0,6,4.0
